In [122]:
import json
import numpy as np
import collections

In [209]:
data = "["
with open("../aminer_papers_0.txt") as myfile:
    counter = 0 
    for line in myfile:
        counter += 1
        data += line + ","
        if counter > 30000:
            break

data = data[:-1] + "]"

In [210]:
json_data = json.loads(data)

In [211]:
len(json_data)

30001

In [219]:
paper_to_authors = dict()
paper_to_journal = dict()

for paper_obj in json_data:
    if 'id' in paper_obj.keys() and 'authors' in paper_obj.keys() and 'venue' in paper_obj.keys() \
        and paper_obj['venue'] in selected_journals:
        paper = paper_obj['id']
        authors = paper_obj['authors']
        journal = paper_obj['venue']
        
        paper_to_authors[paper] = [author['name'] for author in authors]
        paper_to_journal[paper] = journal

In [214]:
len(paper_to_authors)

618

In [215]:
author_to_author_and_papers = dict()

for paper_obj in json_data:
    if 'references' in paper_obj and 'authors' in paper_obj and 'venue' in paper_obj:
        if paper_obj['venue'] in selected_journals:
            print("working")
        else:
            continue
        paper_refs = paper_obj['references']
        author_and_papers = dict()
        for paper_ref in paper_refs:
            if paper_ref in paper_to_authors:
                authors = paper_to_authors[paper_ref]
                for author in authors:
                    author_and_papers[author] = paper_ref
        for author in paper_obj['authors']:
            author_to_author_and_papers[author['name']] = author_and_papers

In [216]:
cleaned_author_to_etc = dict()

for obj in author_to_author_and_papers.keys():
    if author_to_author_and_papers[obj]:
        cleaned_author_to_etc[obj] = author_to_author_and_papers[obj]

In [217]:
# author_to_author_and_papers = cleaned_author_to_etc

In [218]:
len(author_to_author_and_papers)

0

In [187]:
#returns id of author chosen uniformly randomly
def randStartPoint():
    idx = np.random.randint(len(author_to_score))
    return list(author_to_score.keys())[idx]   #perhaps sorting issues here? check later

def getScore(cur_author, proposed_author):
    papers = author_to_author_and_papers[cur_author][proposed_author]
    score = 0
    for paper in papers:
        paper_authors = paper_to_authors[paper]
        avg_author_score = np.mean([author_to_score[auth] for auth in paper_authors])
        score += avg_author_score * paper_to_if[paper]
    
    return score

In [144]:
#to do: sum up multiple papers btwn a and b

#update author_to_score every iteration
def graphtraversal(num_trials, num_walks_per_trial):
    
    output = np.zeros((num_trials, numAuthors))   #matrix of all results. sorted by author id's.
    
    for i in range(num_trials):
        print()
        print("trial number ", str(i))
    
        #select uniform start point
        cur_author = randStartPoint()
        
        print("start at ", cur_author)
        
        for num_walk in range(num_walks_per_trial):
            #calculate next author to go to (propose next state) according to dist. 1/d for current node
            if len(author_to_author_and_papers[cur_author]) == 0: #current author is a dead end
                cur_author = randStartPoint()
                print("start again at ", cur_author)
            else:
                
                idx = np.random.randint(len(author_to_author_and_papers[cur_author]))
                proposed_author = list(author_to_author_and_papers[cur_author].keys())[idx]

                print("proposed author: ", proposed_author)

                #accept or not? according to metropolis hastings 
                pi_y = author_to_score[proposed_author]  #pi(x) = score(x)
                pi_x = author_to_score[cur_author]
                if len(author_to_author_and_papers[proposed_author]) == 0:  #if proposed author is a sink
                    f_y = 1
                else:
                    f_y = 1 / len(author_to_author_and_papers[proposed_author]) #f(x) = 1/d where d is degree of node x (number of outgoing edges)
                if len(author_to_author_and_papers[cur_author]) == 0:
                    f_x = 1
                else:
                    f_x = 1 / len(author_to_author_and_papers[cur_author]) #maybe float errors here
                
                print("pi_y: ", str(pi_y))
                print("pi_x: ", str(pi_x))
                print("f_y: ", str(f_y))
                print("f_x: ", str(f_x))
                print("calculated: ", (pi_y * f_y) / (pi_x * f_x))
                p_accept = min(1, (pi_y * f_y) / (pi_x * f_x))

                print("p accept", str(p_accept))

                if np.random.random() < p_accept:
                    #if accept, update proposed_author's score.  Then move to proposed_author
                    author_to_score[proposed_author] += getScore(cur_author, proposed_author)
                    cur_author = proposed_author
                    
                    print("printing all scores before norm...")
                    for author, score in zip(author_to_score.keys(), author_to_score.values()):
                        print("author: ", author)
                        print("score: ", score)

                    #renormalize everyone author's score
                    norm_score = sum(author_to_score.values())
#                     norm_score = max(author_to_score.values())

                    print("printing all scores...")
                    for author in author_to_score:
                        author_to_score[author] = author_to_score[author] / norm_score
                        print("author: ", author)
                        print("score: ", author_to_score[author])

                else:  #if not accept, randomly start again
                    cur_author = randStartPoint()
                    print("start again at ", cur_author)
        
        #add trial results to output matrix
        sum_score = sum(author_to_score.values())
        output[i] = np.array(list(author_to_score.values()))  #hopefully sorted? check here too
                                                        #don't think this numpy notation works. tweak later lol
     
    avg_output = np.mean(output, axis=0) #columnwise avg for each author
    return output, avg_output

In [148]:
numAuthors = len(author_to_author_and_papers)
startScore = 1
author_to_score = collections.OrderedDict([(author, startScore) for author in author_to_author_and_papers.keys()])

value = 5
paper_to_if = {key:value for key in paper_to_journal.keys()}
output, avg_output = graphtraversal(5, 10)


trial number  0
start at  M. Miura
start again at  Y. Wang
start again at  Yukio Kawanami
start again at  Rodrigo Wagner
start again at  Yasuo Nakamura
start again at  S. J. L. van Eijndhoven
start again at  Olov Richter
start again at  C. Einarsson
start again at  The Netherlands
start again at  Chinlon Lin
start again at  J. M. S. Simões Pereira

trial number  1
start at  Kristin Taraldsen
start again at  Toby Handfield
start again at  Patrick Lincoln
start again at  Z. Sternovsky
start again at  E. Ungeheuer
start again at  Honggao Xu
start again at  V. Andretta
start again at  John A. Campbell
start again at  Katerina Stathi
start again at  J. Griffith
start again at  Gerd Szwillus

trial number  2
start at  Nancy B. Esterly
proposed author:  Jan E. Muhlbauer


KeyError: 'Jan E. Muhlbauer'

In [149]:
author_to_score

OrderedDict([('G. Adam', 1),
             ('K. Schreiber', 1),
             ('R. Farahbod', 1),
             ('V. Gervasi', 1),
             ('U. Glasser', 1),
             ('M. Memon', 1),
             ('Peijuan Wang', 1),
             ('Jiahua Zhang', 1),
             ('Donghui Xie', 1),
             ('Yanyan Xu', 1),
             ('Yun Xu', 1),
             ('D J Lum', 1),
             ('V Upadhyay', 1),
             ('A Smith', 1),
             ('J McFarlane', 1),
             ('Asami Nakai', 1),
             ('Akio Ohtani', 1),
             ('Takeru Ohki', 1),
             ('Toshiko Osada', 1),
             ('Masaharu Iwamoto', 1),
             ('Hiroyuki Hamada', 1),
             ('Nobuo Takeda', 1),
             ('David Galvin', 1),
             ('Pelin Arda-Pirincci', 1),
             ('Sehnaz Bolkent', 1),
             ('Tatsuo Kanda', 1),
             ('Osamu Yokosuka', 1),
             ('Fumio Imazeki', 1),
             ('Hiromitsu Saisho', 1),
             ('Keiichi Nagao',

In [220]:
unique, counts = np.unique(list(paper_to_journal.values()), return_counts = True)
journal_occurences = dict(zip(unique, counts))

In [227]:
min(journal_occurences, key=journal_occurences.get)

'Guide of China Medicine'

In [222]:
journal_occurences['Contemporary Medicine']

422